# 1. Keeping the same parameters

Image size is 1x28x28

      First Convolution : 1x28x28 --> (padding) 1x30x30 --> (kernel & filters) 32x28x28
      Second Convolution : 32x28x28 --> (padding) 32x30x30 --> (kernel & filters) 64x28x28
      After max pooling: 64x14x14
      Third Convolution : 64x14x14 --> (padding) 64x16x16 --> (kernel & filters) 128x14x14
      Forth Convolution : 128x14x14 --> (padding) 128x16x16 --> (kernel & filters) 256x14x14
      After max pooling: 256x7x7
      Fifth Convolution : 256x7x7 --> (kernel & filters) 512x5x5
      sixth Convolution : 512x5x5 --> (kernel & filters) 1024x3x3
      Saventh Convolution : 1024x3x3 --> (kernel & filters) 10x1x1

Using this parameters 

      Train loss = 1.99
      Average Test loss = 1.96 



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
from torchsummary import summary

In [2]:
class Net(nn.Module): # image siE in mnist is 1x28x28
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3,  padding=1)
        self.conv7 = nn.Conv2d(1024, 10, 3)
        self.conv8 = nn.Conv2d(10, 10, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv8(F.relu(self.conv7(x))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 26, 26]          18,496
         MaxPool2d-3           [-1, 64, 13, 13]               0
            Conv2d-4          [-1, 128, 13, 13]          73,856
            Conv2d-5          [-1, 256, 13, 13]         295,168
         MaxPool2d-6            [-1, 256, 6, 6]               0
            Conv2d-7            [-1, 512, 4, 4]       1,180,160
            Conv2d-8           [-1, 1024, 4, 4]       4,719,616
            Conv2d-9             [-1, 10, 2, 2]          92,170
           Conv2d-10             [-1, 10, 1, 1]             410
Total params: 6,380,196
Trainable params: 6,380,196
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.36
Params size (MB): 24.34
Estima

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=1.772858738899231 batch_id=468: 100%|██████████| 469/469 [01:24<00:00,  5.52it/s]



Test set: Average loss: 1.8408, Accuracy: 2105/10000 (21%)



# 2. Changing the parameters ( Decreasing the size of filters)

Image size is 1x28x28

      First Convolution : 1x28x28 --> (padding) 1x30x30 --> (kernel & filters) 1024x28x28
      Second Convolution : 1024x28x28 --> (padding) 1024x30x30 --> (kernel & filters) 512x28x28
      After max pooling: 512x14x14
      Third Convolution : 512x14x14 --> (padding) 512x16x16 --> (kernel & filters) 256x14x14
      Forth Convolution : 256x14x14 --> (padding) 256x16x16 --> (kernel & filters) 128x14x14
      After max pooling: 128x7x7
      Fifth Convolution : 128x7x7 --> (kernel & filters) 64x5x5
      sixth Convolution : 64x5x5 --> (kernel & filters) 32x3x3
      Saventh Convolution : 32x3x3 --> (kernel & filters) 10x1x1

Using this parameters 

      Train loss = 0.37
      Average Test loss = 0.53

### By using this method we can see that there is a decrease in the loss.

In [ ]:
class Net2(nn.Module): # image siE in mnist is 1x28x28
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 1024, 3, padding=1)  
        self.conv2 = nn.Conv2d(1024, 512, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(512, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(128, 64, 3)
        self.conv6 = nn.Conv2d(64, 32, 3)
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device2 = torch.device("cuda" if use_cuda else "cpu")
model2 = Net2().to(device2)
summary(model2, input_size=(1, 28, 28))

In [ ]:
model2 = Net2().to(device2)
optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model2, device2, train_loader, optimizer, epoch)
    test(model2, device2, test_loader)

# 3. Decreasing filters without padding

Image size is 1x28x28

      First Convolution : 1x28x28 --> (kernel & filters) 512x26x26
      Second Convolution : 512x26x26 --> (kernel & filters) 256x24x24
      After max pooling: 256x12x12
      Third Convolution : 256x12x12 --> (kernel & filters) 128x10x10
      Forth Convolution : 128x10x10 --> ((kernel & filters) 64x8x8
      After max pooling: 64x4x4
      Fifth Convolution : 64x4x4 --> (kernel & filters) 32x2x2
      sixth Convolution : 32x2x2 --> (kernel & filters) 10x1x1

Using this parameters 

      Train loss = 1.073
      Average Test loss = 1.176

### By removing padding, loss is increasing, so we will use padding



In [ ]:
class Net3(nn.Module): # image siE in mnist is 1x28x28
    def __init__(self):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(1, 512, 3, padding='valid')  
        self.conv2 = nn.Conv2d(512, 256, 3, padding='valid')
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(256, 128, 3, padding='valid')
        self.conv4 = nn.Conv2d(128, 64, 3, padding='valid')
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(64, 32, 3)
        self.conv6 = nn.Conv2d(32, 10, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device3 = torch.device("cuda" if use_cuda else "cpu")
model3 = Net3().to(device3)
summary(model3, input_size=(1, 28, 28))

In [ ]:
model3 = Net3().to(device3)
optimizer = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model3, device3, train_loader, optimizer, epoch)
    test(model3, device3, test_loader)

# 4. Removing max pooling and kernel size = 5

Image size is 1x28x28

      First Convolution : 1x28x28 --> (padding) 1x30x30 --> (kernel & filters) 1024x28x28
      Second Convolution : 1024x28x28 --> (padding) 1024x30x30 --> (kernel & filters) 512x28x28
      Third Convolution : 512x28x28--> (padding) 512x30x30 --> (kernel & filters) 256x28x28
      Forth Convolution : 256x28x28 --> (padding) 256x30x30 --> (kernel & filters) 128x28x28
      Fifth Convolution : 128x28x28 --> (kernel & filters) 64x24x24
      sixth Convolution : 64x24x24 --> (kernel & filters) 32x20x20
      Saventh Convolution : 32x20x20 --> (kernel & filters) 10x16x16
      Eight Convolution : 10x16x16 --> (kernel & filters) 10x12x12
      Ninth Convolution : 10x12x12 --> (kernel & filters) 10x8x8
      Tenth Convolution : 10x8x8 --> (kernel & filters) 10x4x4
      Tenth Convolution : 10x4x4 --> (kernel & filters) 10x1x1

Using this parameters 

      Train loss = 2.30
      Average Test loss = 2.30

### By removing pooling loss is incresing

In [ ]:
class Net4(nn.Module): # image siE in mnist is 1x28x28
    def __init__(self):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(1, 1024, 5, padding=2)  
        self.conv2 = nn.Conv2d(1024, 512, 5, padding=2)
        self.conv3 = nn.Conv2d(512, 256, 5, padding=2)
        self.conv4 = nn.Conv2d(256, 128, 5, padding=2)
        self.conv5 = nn.Conv2d(128, 64, 5)
        self.conv6 = nn.Conv2d(64, 32, 5)
        self.conv7 = nn.Conv2d(32, 10, 5)
        self.conv8 = nn.Conv2d(10, 10, 5)
        self.conv9 = nn.Conv2d(10, 10, 5)
        self.conv10 = nn.Conv2d(10, 10, 5)
        self.conv11 = nn.Conv2d(10, 10, 4)

    def forward(self, x):
        x = F.relu(self.conv2(F.relu(self.conv1(x)))) 
        x = F.relu(self.conv4(F.relu(self.conv3(x))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv8(F.relu(self.conv7(x))))
        x = F.relu(self.conv10(F.relu(self.conv9(x))))
        x = F.relu(self.conv11(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
use_cuda = torch.cuda.is_available()
device4 = torch.device("cuda" if use_cuda else "cpu")
model4 = Net4().to(device4)
summary(model4, input_size=(1, 28, 28))

In [ ]:
model4 = Net4().to(device4)
optimizer = optim.SGD(model4.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model4, device3, train_loader, optimizer, epoch)
    test(model4, device3, test_loader)